In [2]:
import os

db_dir = os.environ["DATA"] + "PatImgXAI_data/db2.0.0/"
datasets_dir_path = os.environ["DATA"] + "PatImgXAI_data/db2.0.0/datasets/03_protov4/"
model_dir_root = os.environ["DATA"] + "models/db2.0.0/03_protov4/"

shap_scale_img_path = os.path.join(db_dir,"shap_scale.png")
yes_pred_img_path = os.path.join(db_dir,"button_yes.png")
no_pred_img_path = os.path.join(db_dir,"button_no.png")
yes_small_pred_img_path = os.path.join(db_dir,"button_yes_small.png")
no_small_pred_img_path = os.path.join(db_dir,"button_no_small.png")
pos_pred_legend_path = os.path.join(db_dir,"cf_info_pos.png")
neg_pred_legend_path = os.path.join(db_dir,"cf_info_neg.png")
interface_dir = os.environ["DATA"] + "webinterfaces/int04_prototype/"

XAI_DATASET_SIZE = 100

N_JOBS = 20
N_JOBS_GPU = 6

In [3]:
# Number of images generated
NBGEN = 1000000

# Grid division of each image
X_DIVISIONS = 6
Y_DIVISIONS = 6

# Size of the images in pixels
img_size = (700, 700)

# Probability to generate a geometrical shape at each position in the grid
SHAPE_PROB = 0.5

# Define available shapes
SHAPES = ['circle', 'square', 'triangle']
COLORS  = ["#A33E9A", "#E0B000", "#0C90C0"]

explict_colors_dict = {
    "#A33E9A": "purple",
    "#E0B000": "yellow",
    "#0C90C0": "blue"
}

In [4]:
from xaipatimg.datagen.gendataset import generic_rule_exist_row_with_only_shape, generic_rule_N_times_color_exactly, \
    generic_rule_shape_color_plus_shape_equals_N, generic_rule_shape_in_every_row, generic_rule_shape_color_times_2_shape_equals_shape

rules_data = [
    {"name": "disc_1_triangle_all", "gen_fun": generic_rule_shape_in_every_row, "gen_kwargs": {"shape": "triangle", "y_division": Y_DIVISIONS}, "question": "In the image, is there a triangle in every row (1, ..., 6)?", "target_acc" : 1.0, "shown_acc" : 1.0, "samples_interface": 3, "pos_llm_scaffold": "The AI predicts |YES| because every row contains at least one triangle : \n - Row 1 : XX, XX, XX\n- Row 2 : XX, XX, XX\n- Row 3 : XX, XX, XX\n- Row 4 : XX, XX, XX\n- Row 5 : XX, XX, XX\n- Row 5 : XX, XX, XX", "neg_llm_scaffold": "The AI predicts |NO| because the rows X and X do not contain any triangle."},

    {"name": "disc_1_triangle_all_2", "gen_fun": generic_rule_shape_in_every_row, "gen_kwargs": {"shape": "triangle", "y_division": Y_DIVISIONS}, "question": "In the image, is there a triangle in every row (1, ..., 6)?", "target_acc" : 1.0, "shown_acc" : 1.0, "samples_interface": 5, "pos_llm_scaffold": "The AI predicts |YES| because every row contains at least one triangle : \n - Row 1 : XX, XX, XX\n- Row 2 : XX, XX, XX\n- Row 3 : XX, XX, XX\n- Row 4 : XX, XX, XX\n- Row 5 : XX, XX, XX\n- Row 5 : XX, XX, XX", "neg_llm_scaffold": "The AI predicts |NO| because the rows X and X do not contain any triangle."},

    {"name": "disc_1_triangle_all_3", "gen_fun": generic_rule_shape_in_every_row, "gen_kwargs": {"shape": "triangle", "y_division": Y_DIVISIONS}, "question": "In the image, is there a triangle in every row (1, ..., 6)?", "target_acc" : 1.0, "shown_acc" : 1.0, "samples_interface": 5, "pos_llm_scaffold": "The AI predicts |YES| because every row contains at least one triangle : \n - Row 1 : XX, XX, XX\n- Row 2 : XX, XX, XX\n- Row 3 : XX, XX, XX\n- Row 4 : XX, XX, XX\n- Row 5 : XX, XX, XX\n- Row 5 : XX, XX, XX", "neg_llm_scaffold": "The AI predicts |NO| because the rows X and X do not contain any triangle."},


    {"name": "easy_1_6_blue", "gen_fun": generic_rule_N_times_color_exactly, "gen_kwargs": {"color": "#0C90C0", "N": 6, "x_division": X_DIVISIONS, "y_division": Y_DIVISIONS}, "question": "In the image, are there exactly 6 blue symbols?", "target_acc": 0.9, "shown_acc": 0.85, "samples_interface": 7, "pos_llm_scaffold": "The AI predicts |YES| because there is exactly 6 blue symbols, which are located at :\n- XX\n- XX\n- XX\n- XX\n- XX\n- XX", "neg_llm_scaffold": "The AI predicts |NO| because there is X blue symbols instead of 6. They are located at : \n- XX\n- XX\n- XX\n- XX\n- XX."},

    {"name": "easy_3_7_purple", "gen_fun": generic_rule_N_times_color_exactly, "gen_kwargs": {"color": "#A33E9A", "N": 7, "x_division": X_DIVISIONS, "y_division": Y_DIVISIONS}, "question": "In the image, are there exactly 7 purple symbols?", "target_acc": 0.9, "shown_acc": 0.85, "samples_interface": 14, "pos_llm_scaffold": "The AI predicts |YES| because there is exactly 7 purple symbols, which are located at :\n- XX\n- XX\n- XX\n- XX\n- XX\n- XX\n- XX", "neg_llm_scaffold": "The AI predicts |NO| because there is X purple symbols instead of 7. They are located at : \n- XX\n- XX\n- XX\n- XX\n- XX\n- XX."},

    {"name": "easy_4_row_triangle", "gen_fun": generic_rule_exist_row_with_only_shape, "gen_kwargs": {"shape": "triangle", "y_division": Y_DIVISIONS},
     "question": "In the image, is there at least one row (1, ..., 6) containing only triangles?", "target_acc": 0.9, "shown_acc": 0.85, "samples_interface": 14, "pos_llm_scaffold": "The AI predicts |YES| because there is at least one row which contains only triangles : \nRow X contains only triangles which are located at XX, XX, XX", "neg_llm_scaffold": "The AI predicts |NO| because there is not a single row containing only triangles :\nRow 1 contains a non-triangle symbol at XX\nRow 2 contains non-triangle symbols at XX, XX, XX.\nRow 3 does not contain any symbol at all\n ..."},

    {"name": "easy_5_5_yellow", "gen_fun": generic_rule_N_times_color_exactly, "gen_kwargs": {"color": "#E0B000", "N": 5, "x_division": X_DIVISIONS, "y_division": Y_DIVISIONS}, "question": "In the image, are there exactly 5 yellow symbols?", "target_acc": 0.9, "shown_acc": 0.85, "samples_interface": 14,  "pos_llm_scaffold": "The AI predicts |YES| because there is exactly 5 yellow symbols, which are located at :\n- XX\n- XX\n- XX\n- XX\n- XX", "neg_llm_scaffold": "The AI predicts |NO| because there is X yellow symbols instead of 5, which are located at : \n- XX\n- XX\n- XX\n- XX\n- XX\n- XX."},

    {"name": "easy_6_row_square", "gen_fun": generic_rule_exist_row_with_only_shape, "gen_kwargs": {"shape": "square", "y_division": Y_DIVISIONS},
     "question": "In the image, is there at least one row (1, ..., 6) containing only squares?", "target_acc": 0.9, "shown_acc": 0.85, "samples_interface": 14, "pos_llm_scaffold": "The AI predicts |YES| because there is at least one row which contains only squares : \nRow X contains only squares which are located at XX, XX, XX", "neg_llm_scaffold": "The AI predicts |NO| because there is not a single row containing only squares :\nRow 1 contains a non-square symbol at XX\nRow 2 contains non-square symbols at XX, XX, XX.\nRow 3 does not contain any symbol at all\n ..."},


    {"name": "hard_1_blue_square_plus_circle_8", "gen_fun": generic_rule_shape_color_plus_shape_equals_N, "gen_kwargs": {"color1": "#0C90C0", "shape1": "square", "shape2": "circle", "N": 8, "x_division": X_DIVISIONS, "y_division": Y_DIVISIONS,},
     "question": "In the image, does the number of blue squares plus (+) the number of circles equal to 8?", "question_llm": "In the image, does the number of blue squares plus (+) the number of circles of any color equal to 8", "target_acc": 0.9, "shown_acc": 0.85, "samples_interface": 7, "pos_llm_scaffold": "The AI predicts |YES| because \n\n There is a total of X blue squares at positions : \n- XX\n- XX\n- XX\n- XX\n \nThere is a total of X circles at positions : \n- XX\n- XX\n- XX\n- XX\n\n X + X = 8", "neg_llm_scaffold": "The AI predicts |NO| because \n\n There is a total of X blue squares at positions : \n- XX\n- XX\n- XX\n- XX\n \nThere is a total of X circles at positions : \n- XX\n- XX\n- XX\n- XX\n X + X = X ≠ 8"},

    {"name": "hard_3_yellow_circle_plus_triangle_9", "gen_fun": generic_rule_shape_color_plus_shape_equals_N, "gen_kwargs": {"color1": "#E0B000", "shape1": "circle", "shape2": "triangle", "N": 9, "x_division": X_DIVISIONS, "y_division": Y_DIVISIONS},
     "question": "In the image, does the number of yellow circles plus (+) the number of triangles equal to 9?",
     "question_llm": "In the image, does the number of yellow circles plus (+) the number of triangles of any color equal to 9?","target_acc": 0.9, "shown_acc": 0.85, "samples_interface": 14, "pos_llm_scaffold": "The AI predicts |YES| because \n\n There is a total of X yellow circles at positions : \n- XX\n- XX\n- XX\n- XX\n \nThere is a total of X triangles at positions : \n- XX\n- XX\n- XX\n- XX\n\n X + X = 9", "neg_llm_scaffold": "The AI predicts |NO| because \n\n There is a total of X yellow circles at positions : \n- XX\n- XX\n- XX\n- XX\n \nThere is a total of X triangles at positions : \n- XX\n- XX\n- XX\n- XX\n X + X = X ≠ 9"},
    #
    {"name": "hard_4_purple_squares_times2_circles", "gen_fun": generic_rule_shape_color_times_2_shape_equals_shape, "gen_kwargs": {"color1": "#A33E9A", "shape1": "square", "shape2": "circle", "x_division": X_DIVISIONS, "y_division": Y_DIVISIONS,},
     "question": "In the image, does the number of purple squares multiplied by 2 (×2) equal to the number of circles?",
     "question_llm": "In the image, does the number of purple squares multiplied by 2 (×2) equal to the number of circles of any color?", "target_acc": 0.9, "shown_acc": 0.85, "samples_interface": 14, "pos_llm_scaffold": "The AI predicts |YES| because \n\n There is a total of X purple squares at positions : \n- XX\n- XX\n- XX\n- XX\n \nThere is a total of X circles at positions : \n- XX\n- XX\n- XX\n- XX\n\n X × 2 = X", "neg_llm_scaffold": "There is a total of X purple squares at positions : \n- XX\n- XX\n- XX\n- XX\n \nThere is a total of X circles at positions : \n- XX\n- XX\n- XX\n- XX\n\n X × 2 = X ≠ X"},

    {"name": "hard_5_purple_triangle_plus_square_7", "gen_fun": generic_rule_shape_color_plus_shape_equals_N, "gen_kwargs": {"color1": "#A33E9A", "shape1": "triangle", "shape2": "square", "N": 7, "x_division": X_DIVISIONS, "y_division": Y_DIVISIONS},
     "question": "In the image, does the number of purple triangles plus (+) the number of squares equal to 7?",
     "question_llm": "In the image, does the number of purple triangles plus (+) the number of squares of any color equal to 7?", "target_acc": 0.9, "shown_acc": 0.85, "samples_interface": 14, "pos_llm_scaffold": "The AI predicts |YES| because \n\n There is a total of X purple triangles at positions : \n- XX\n- XX\n- XX\n- XX\n \nThere is a total of X squares at positions : \n- XX\n- XX\n- XX\n- XX\n\n X + X = 7", "neg_llm_scaffold": "The AI predicts |NO| because \n\n There is a total of X purple triangles at positions : \n- XX\n- XX\n- XX\n- XX\n \nThere is a total of X squares at positions : \n- XX\n- XX\n- XX\n- XX\n X + X = X ≠ 7"},

    {"name": "hard_6_blue_circles_times2_triangles", "gen_fun": generic_rule_shape_color_times_2_shape_equals_shape, "gen_kwargs": {"color1": "#0C90C0", "shape1": "circle", "shape2": "triangle", "x_division": X_DIVISIONS, "y_division": Y_DIVISIONS,},
     "question": "In the image, does the number of blue circles multiplied by 2 (×2) equal to the number of triangles?", "question_llm": "In the image, does the number of blue circles multiplied by 2 (×2) equal to the number of triangles of any color?","target_acc": 0.9, "shown_acc": 0.85, "samples_interface": 14, "pos_llm_scaffold": "The AI predicts |YES| because \n\n There is a total of X blue circles at positions : \n- XX\n- XX\n- XX\n- XX\n \nThere is a total of X triangles at positions : \n- XX\n- XX\n- XX\n- XX\n\n X × 2 = X", "neg_llm_scaffold": "There is a total of X blue circles at positions : \n- XX\n- XX\n- XX\n- XX\n \nThere is a total of X triangles at positions : \n- XX\n- XX\n- XX\n- XX\n\n X × 2 = X ≠ X"},
]


In [5]:
from xaipatimg.datagen.dbimg import load_db

db = load_db(db_dir)

In [6]:
from xaipatimg.ml.xai import generate_shap_resnet18, generate_counterfactuals_resnet18_random_approach, \
    create_xai_index, generate_cam_resnet18
from tqdm import tqdm

for rule_idx in tqdm(range(len(rules_data))):

    model_dir = os.path.join(model_dir_root, rules_data[rule_idx]["name"])
    dataset_filename = rules_data[rule_idx]["name"] + "_test.csv"
    generic_rule_fun = rules_data[rule_idx]["gen_fun"]
    generic_rule_fun_kwargs = rules_data[rule_idx]["gen_kwargs"]
    xai_output_paths = {
        "shap" : "shap",
        "cf" : "cf"
    }

    # generate_shap_resnet18(db_dir, datasets_dir_path=datasets_dir_path, dataset_filename=dataset_filename,
    #                        model_dir=model_dir, xai_output_path=os.path.join(model_dir, xai_output_paths["shap"]),
    #                        yes_pred_img_path=yes_pred_img_path, no_pred_img_path=no_pred_img_path, device="cuda:0", n_jobs=N_JOBS,
    #                        dataset_size=XAI_DATASET_SIZE, masker="ndarray", shap_scale_img_path=shap_scale_img_path)
    #
    # generate_counterfactuals_resnet18_random_approach(db_dir, datasets_dir_path=datasets_dir_path, dataset_filename=dataset_filename,
    #                                                   model_dir=model_dir,
    #                                                   xai_output_path=os.path.join(model_dir, xai_output_paths["cf"]),
    #                                                   yes_pred_img_path=yes_pred_img_path, no_pred_img_path=no_pred_img_path,
    #                                                   shapes=SHAPES, colors=COLORS, empty_probability=1-SHAPE_PROB,
    #                                                   max_depth=10, nb_tries_per_depth=2000, generic_rule_fun=generic_rule_fun,
    #                                                   devices=["cuda:0", "cuda:1"], n_jobs=N_JOBS_GPU,
    #                                                   dataset_size=XAI_DATASET_SIZE,pos_pred_legend_path=pos_pred_legend_path,
    #                                                   neg_pred_legend_path=neg_pred_legend_path,
    #                                                   **generic_rule_fun_kwargs)

    create_xai_index(db_dir, datasets_dir_path=datasets_dir_path, dataset_filename=dataset_filename, model_dir=model_dir,
                     xai_dirs=xai_output_paths, dataset_size=XAI_DATASET_SIZE, device="cuda:0")


/home/docker/anaconda3/envs/Pytorch_exp/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
  0%|          | 0/13 [00:00<?, ?it/s]

Loading dataset content for disc_1_triangle_all_test.csv



100%|██████████| 100/100 [00:01<00:00, 60.88it/s][A
Using cache found in /home/docker/.cache/torch/hub/pytorch_vision_v0.10.0
/home/docker/anaconda3/envs/Pytorch_exp/lib/python3.12/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/docker/anaconda3/envs/Pytorch_exp/lib/python3.12/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)
  8%|▊         | 1/13 [00:02<00:33,  2.79s/it]

Loading dataset content for disc_1_triangle_all_2_test.csv



100%|██████████| 100/100 [00:01<00:00, 61.09it/s][A
Using cache found in /home/docker/.cache/torch/hub/pytorch_vision_v0.10.0
/home/docker/anaconda3/envs/Pytorch_exp/lib/python3.12/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/docker/anaconda3/envs/Pytorch_exp/lib/python3.12/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)
 15%|█▌        | 2/13 [00:05<00:27,  2.46s/it]

...


100%|██████████| 100/100 [00:01<00:00, 66.55it/s][A
Using cache found in /home/docker/.cache/torch/hub/pytorch_vision_v0.10.0
/home/docker/anaconda3/envs/Pytorch_exp/lib/python3.12/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/docker/anaconda3/envs/Pytorch_exp/lib/python3.12/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)
 85%|████████▍ | 11/13 [00:24<00:04,  2.08s/it]

Loading dataset content for hard_5_purple_triangle_plus_square_7_test.csv



100%|██████████| 100/100 [00:01<00:00, 62.28it/s][A
Using cache found in /home/docker/.cache/torch/hub/pytorch_vision_v0.10.0
/home/docker/anaconda3/envs/Pytorch_exp/lib/python3.12/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/docker/anaconda3/envs/Pytorch_exp/lib/python3.12/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)
 92%|█████████▏| 12/13 [00:26<00:02,  2.11s/it]

Loading dataset content for hard_6_blue_circles_times2_triangles_test.csv



100%|██████████| 100/100 [00:01<00:00, 64.99it/s][A
Using cache found in /home/docker/.cache/torch/hub/pytorch_vision_v0.10.0
/home/docker/anaconda3/envs/Pytorch_exp/lib/python3.12/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/docker/anaconda3/envs/Pytorch_exp/lib/python3.12/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)
100%|██████████| 13/13 [00:28<00:00,  2.17s/it]


In [5]:
from transformers import AutoTokenizer
from transformers import AutoModelForCausalLM
import csv
from xaipatimg.ml.xai import generate_LLM_explanations, create_xai_index
from tqdm import tqdm

model_id = "openai/gpt-oss-20b"
tokenizer = AutoTokenizer.from_pretrained(model_id)
llm_model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto",
    torch_dtype="auto",
)

for rule_idx in tqdm(range(len(rules_data))):

    model_dir = os.path.join(model_dir_root, rules_data[rule_idx]["name"])
    dataset_filename = rules_data[rule_idx]["name"] + "_test.csv"

    # Extracting the subset of indices of samples selected for the experimental interface, in order to ease the cost of calculation
    interface_content_path = os.path.join(interface_dir, "res", "tasks", f"{rules_data[rule_idx]["name"]}_content.csv")
    interface_selected_idx = [int(row["og_idx"]) for row in list(csv.DictReader(open(interface_content_path), delimiter=','))]

    xai_output_paths = {
        "shap" : "shap",
        "cf" : "cf",
        "llm" : "llm",
    }
    generate_LLM_explanations(db_dir, db, datasets_dir_path=datasets_dir_path, dataset_filename=dataset_filename,
                              model_dir=model_dir, llm_model=llm_model, llm_tokenizer=tokenizer,
                              xai_output_path=os.path.join(model_dir, xai_output_paths["llm"]),
                              explicit_colors_dict=explict_colors_dict, question=rules_data[rule_idx]["question"],
                              yes_pred_img_path=yes_pred_img_path, no_pred_img_path=no_pred_img_path,
                              yes_pred_img_path_small=yes_small_pred_img_path, no_pred_img_path_small=no_small_pred_img_path,
                              device="cuda:0", dataset_size=XAI_DATASET_SIZE, only_for_index=interface_selected_idx,
                              path_to_counterfactuals_dir_for_model_errors=os.path.join(model_dir, xai_output_paths["cf"]),
                              pos_llm_scaffold=rules_data[rule_idx]["pos_llm_scaffold"],
                              neg_llm_scaffold=rules_data[rule_idx]["neg_llm_scaffold"])

    create_xai_index(db_dir, datasets_dir_path=datasets_dir_path, dataset_filename=dataset_filename, model_dir=model_dir,
                     xai_dirs=xai_output_paths,
                     dataset_size=XAI_DATASET_SIZE,device="cuda:0")


/home/docker/anaconda3/envs/Pytorch_exp/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
`torch_dtype` is deprecated! Use `dtype` instead!
  0%|          | 0/1 [00:00<?, ?it/s]

/home/docker/data/models/db2.0.0/02_protov3/hard_6_blue_circles_times2_triangles/llm
Loading dataset content for hard_6_blue_circles_times2_triangles_test.csv



100%|██████████| 100/100 [00:01<00:00, 62.88it/s]
Using cache found in /home/docker/.cache/torch/hub/pytorch_vision_v0.10.0
/home/docker/anaconda3/envs/Pytorch_exp/lib/python3.12/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/docker/anaconda3/envs/Pytorch_exp/lib/python3.12/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)

100%|██████████| 10/10 [30:00<00:00, 180.01s/it]


Loading dataset content for hard_6_blue_circles_times2_triangles_test.csv



100%|██████████| 100/100 [00:01<00:00, 61.53it/s][A
Using cache found in /home/docker/.cache/torch/hub/pytorch_vision_v0.10.0
/home/docker/anaconda3/envs/Pytorch_exp/lib/python3.12/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/docker/anaconda3/envs/Pytorch_exp/lib/python3.12/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)
100%|██████████| 1/1 [30:04<00:00, 1804.54s/it]
